In [143]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from math import sqrt, modf, cos, sin
from haversine import haversine
from tqdm import tqdm_notebook
tqdm.pandas(tqdm_notebook())
from math import pi
import pyproj


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [2]:
df = pd.read_pickle('..\\..\\notGitHub\\partials\\DC_pulled_validated.pickle')

df.columns = ["lon", "lat", "unit_num", "loc_str", "request", "Info"]

In [3]:
df.head()

,lon,lat,unit_num,loc_str,request,Info
ADDRESS_ID,,,,,,
243225,-77.037265,38.936436,1181.0,"38.93643575527931,-77.03726510819578",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.936...
284325,-77.024650,38.920318,917.0,"38.92031826898436,-77.02464966400156",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.920...
294461,-77.081959,38.935867,671.0,"38.93586687687617,-77.08195893397365",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.935...
235844,-77.035880,38.923626,664.0,"38.92362582942218,-77.03588045315645",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.923...
220630,-77.061250,38.943732,638.0,"38.94373173501572,-77.06125006503876",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.943...


In [4]:
# df.head(1).Info.values[0]['data']['mobilityScore']

In [5]:
"""The current json field queries are incorrect so this will be left so the correct values can be substituted in later."""

# string for the json structure, i.e. where I am looking for the scores
s1 = "data"
s2 = "mobilityScore"
# first field, i.e. individual score entry
f1 = "score"
# string for the structure to the other fields
s3 = "scoreBreakdown"
# other fields
f2 = "bikeshare"
f3 = "carshare"
f4 = "masstransit"
f5 = "ridehailing"

In [6]:
df.head(1).Info.values[0][s1][s2][f1],df.head(1).Info.values[0][s1][s2][s3][f2]

(77, '0%')

In [7]:
df['total_score'] = df.Info.apply(lambda x: x[s1][s2][f1])
df['bike_score'] = df.Info.apply(lambda x: float(x[s1][s2][s3][f2].strip('%')))
df['car_score'] = df.Info.apply(lambda x: float(x[s1][s2][s3][f3].strip('%')))
df['mass_score'] = df.Info.apply(lambda x: float(x[s1][s2][s3][f4].strip('%')))
df['ride_score'] = df.Info.apply(lambda x: float(x[s1][s2][s3][f5].strip('%')))

In [8]:
df.head()

,lon,lat,unit_num,loc_str,request,Info,total_score,bike_score,car_score,mass_score,ride_score
ADDRESS_ID,,,,,,,,,,,
243225,-77.037265,38.936436,1181.0,"38.93643575527931,-77.03726510819578",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.936...,77,0.0,0.0,74.0,26.0
284325,-77.024650,38.920318,917.0,"38.92031826898436,-77.02464966400156",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.920...,83,18.0,4.0,57.0,21.0
294461,-77.081959,38.935867,671.0,"38.93586687687617,-77.08195893397365",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.935...,62,0.0,24.0,26.0,50.0
235844,-77.035880,38.923626,664.0,"38.92362582942218,-77.03588045315645",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.923...,93,20.0,4.0,61.0,15.0
220630,-77.061250,38.943732,638.0,"38.94373173501572,-77.06125006503876",https://mobilityscore.transitscreen.io/api/v1/...,{'data': {'mobilityScore': {'latitude': 38.943...,69,23.0,18.0,23.0,36.0


In [172]:
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

############
# These are functions that will be used in processing the points into hexagonal shapes and creating the shapefiles of those hexes
############

def hex_round(x,y):
    z = -x -y
    x_d, x = modf(x)
    y_d, y = modf(y)
    z_d, z = modf(z)
    
    if x_d>y_d and x_d>z_d:
        x = -y-z
    elif y_d>z_d:
        y = -x-z
    else:
        z=-x-y
        
    return (int(x),int(y),int(z))

sr3 = sqrt(3)/3

def point_loc_to_hex_loc(x_h, y_h, size):
    x = (x_h * 2/3)/ size
    y = ((-x_h/3) + (sr3 * y_h)) / size
    return hex_round(x,y)

def hex_center(x_row, y_row, size):
    x = size * 3/2 * x_row
    y = size * (sr3) * (y_row + x_row/2)
    return x, y

# this simple little script just uses the above to find which hex a point falls within
hex_assign = lambda p: point_loc_to_hex_loc(p.x-washington_x, p.y-washington_y, hex_size)

# this simple little script just takes the index and makes a unique key
index_key = lambda t, m: (t[0]+m+1 + (t[1]+m+1)*2*m)

############
# The city grid object is the thing that stores all the information for a single city.
# It processes dataframes into the format that will work with Bokeh.
############


class city_grid:
    def __init__(self,name, lat, lon, grid_rings, hex_size):
        self.name = name
        self.lat = lat
        self.lon = lon
        
        self.X, self.Y = pyproj.transform(wgs84, webdb, self.lon, self.lat)
        
        self.grid_rings = grid_rings # how many hexes there are from middle ot edge of grid
        self.lat_size = hex_size # the size of each individual hex, center to vertex in degrees of latitude
        self.hex_meters = (hex_size * 110574.0) # at the local level I should use meters not degrees
        
        # 1 decimal degree of latitude = 110.574 kms.  Longitude varies so I use latitude for consistency.
        self.points = pd.DataFrame(columns = ['hex', 'hex_key','lat','lon', 'X', 'Y','unit_num', 'total_score',
                                    'bike_score','car_score','mass_score','ride_score'])
        self.hexes = pd.DataFrame(columns = ['grid', 'hex_key','x_shape','y_shape', 'point_list',
                                             'total_score','bike_score','car_score',
                                             'mass_score','ride_score','points_list'])
        
    ########
    # Code to process points in lat-long and assign to hexes
    ########
        
    def point_frame_entry(self, df):
        for col in self.points.columns:
            if col in df.columns:
                self.points[col] = df[col]
            else:
                self.points[col] = None
    
            
    def kilometer_conversion(self):
        self.points['X'] = None
        self.points['Y'] = None
        for row in self.points[['lat','lon']].itertuples():
            x, y = pyproj.transform(wgs84, webdb, row[2], row[1])
            x = x- self.X
            y = y- self.Y
            
            self.points.at[row[0], 'X'] = x
            self.points.at[row[0], 'Y'] = y


    def hex_clear_and_assign(self):
        self.points['hex'] = None
        t = tqdm_notebook(self.points[['X','Y']].itertuples(), total = self.points[['X','Y']].shape[0])
        print("Calculating hex assignments for points")
        
        for row in t:
            index = point_loc_to_hex_loc( (row[1])  , (row[2]), self.hex_meters)
            self.points.at[row[0],'hex'] = index
            self.points.at[row[0],'hex_key'] = index_key(index, self.grid_rings)

        # This returns the "index" of the hexagon where the point lies.  It's given in hexagonal 
        # coordinates i.e. 3 axis in a plane not two (see below).  This is calculated algebraically 
        # from the size of the hexagons with no reference to the hexagon shapes themselves, those will
        # be generated later

    ###########
    # Code to generate grid of hexagons
    ###########
            """this setup uses a 3 coordinate layout for a plane
        instead of x and y indicating directions seperated by 90 degrees
        x, y and z are seperated by 60 degrees.  This layout makes it easier
        to track hexagonal calculations because any adjacent hexagon is always
        exactly 1 away along two of the tree axis.  E.G. 1, 0, -1 is next to 
        2, -1, -1 and 0, 0, 0."""
            
            
            
    def make_ring(self,n):
        ring = []
        for m in range(1,n+1):
            ring.append((n,-m,-n+m))
            ring.append((-m,-n+m,n))
            ring.append((-n+m,n,-m))
            ring.append((-n,m, n-m))
            ring.append((m,n-m,-n))
            ring.append((n-m,-n,m))
        return ring
    
    def fill_grid(self):
        hex_list = [(0,0,0)]
        for n in range(1,self.grid_rings+1):
            hex_list+=self.make_ring(n)
        return hex_list
        
    def yx_generator(self, grid):
        # used by hex_outline_creator
        size = self.hex_meters
        cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + self.Y
        cx = 1.5 * float(grid[0]) * size + self.X
        Y=[]
        X=[]
        for n in range(6):
            angle=n*pi/3
            Y.append(size*sin(angle)+cy)            
            X.append(size*cos(angle)+cx)
        return Y, X

    def grid_creation(self):

        g = self.fill_grid()
        l = len(g)
        print(l)
        s= self.grid_rings
        print("Creating dataframe and id-coding hexagons")
        columns = ['total_score','bike_score','car_score',
             'mass_score','ride_score','points_list' ]
        self.hexes = pd.DataFrame(zeros((l,1)), columns=['grid'])
        self.hexes['grid'] = g
        
        self.hexes['hex_key'] = self.hexes.grid.apply(lambda x: index_key(x, s))
        
        for col in columns:
            self.hexes[col] = pd.np.empty((len(self.hexes), 0)).tolist()
            
        YX = self.hexes['grid'].progress_apply(self.yx_generator)
        self.hexes['Y'] = YX.apply(lambda x: x[0])
        self.hexes['X'] = YX.apply(lambda x: x[1])
        self.hexes.set_index('hex_key', inplace=True)
            
    def hex_pairing(self):
        # This uses the fact that every point already has calculated the "id" of the hex in which it falls, using it's coordinates.
        points = self.points
        hexes = self.hexes        
        points['point_id'] = points.index
        t = tqdm_notebook(points.itertuples(), total=points.shape[0])
        print("Generating list of values within each hex")
        
        for row in t:
            hex_key = row[2] # this is the id of the hex it matches
            
            hex_row = hexes.loc[hex_key]            
            hex_row['total_score'].append(row[8])
            hex_row['bike_score'].append(row[9])
            hex_row['car_score'].append(row[10])
            hex_row['mass_score'].append(row[11])
            hex_row['ride_score'].append(row[12])
            hex_row['points_list'].append(row[13])      
            
            
#             hex_row['total_score'].apply(lambda l: l.append(row.total_score))
#             hex_row['bike_score'].apply(lambda l: l.append(row.bike_score))
#             hex_row['car_score'].apply(lambda l: l.append(row.car_score))
#             hex_row['mass_score'].apply(lambda l: l.append(row.mass_score))
#             hex_row['ride_score'].apply(lambda l: l.append(row.ride_score))
#             hex_row['points_list'].apply(lambda l: l.append(row.point_id))            
            
#             hexes.loc[id, 'total_score'].apply(lambda l: l.append(row.total_score))
#             hexes.loc[id, 'bike_score'].apply(lambda l: l.append(row.bike_score))
#             hexes.loc[id, 'car_score'].apply(lambda l: l.append(row.car_score))
#             hexes.loc[id, 'mass_score'].apply(lambda l: l.append(row.mass_score))
#             hexes.loc[id, 'ride_score'].apply(lambda l: l.append(row.ride_score))
#             hexes.loc[id, 'points_list'].apply(lambda l: l.append(row.point_id))
            
        print("Done")
        
    def export_frame(self):
        fp = self.name + ".pickle"
        self.hexes.to_pickle(fp)

In [ ]:
grid_rings = 200
city_radius = 0.20 # in lat-long degrees
hex_size = city_radius/(2*grid_rings)

washington_y = 38.904978
washington_x = -77.039658

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator


Washington = city_grid("Washington", washington_y, washington_x, grid_rings, hex_size)

Washington.point_frame_entry(df)
Washington.kilometer_conversion()
Washington.hex_clear_and_assign()
Washington.grid_creation()
Washington.points.head()
Washington.hex_pairing()
Washington.export_frame()

HBox(children=(IntProgress(value=0, max=21126), HTML(value='')))

Calculating hex assignments for points
120601
Creating dataframe and id-coding hexagons


100%|███████████████████████████████████████████████████████████████████████| 120601/120601 [00:02<00:00, 47264.43it/s]


HBox(children=(IntProgress(value=0, max=21126), HTML(value='')))

Generating list of values within each hex


In [13]:
#
# Speedtesting, fun!
#


# def iat_assign():
#     df['XY'] = None
#     l = df.shape[0]
#     for i in range(l):
#         df.iat[i,11] = (df.iat[i,1],df.iat[i,0])
        
# def at_assign():
#     df['XY'] = None
#     for i in df.index:
#         df.at[i,'XY'] = (df.at[i,'lon'], df.at[i,'lat'])

# def itertuples_at_assign():
#     df['XY'] = None
#     for row in df[['lon','lat']].itertuples():
#         XY = (row[2],row[1])
#         df.at[row[0], 'XY'] = XY
        
# def itertuples_at_assign_2():
#     df['XY'] = None
#     for row in df[['lon','lat']].itertuples():
#         df.at[row[0], 'XY'] = (row[2],row[1])
        
# def itertuples_iat_assign():
#     df['XY'] = None
#     for row in df[['lon','lat']].reset_index().itertuples():
#         XY = (row[2],row[1])
#         df.iat[row[0], 11] = XY
        
# %timeit iat_assign()
# %timeit at_assign()
# %timeit itertuples_at_assign()
# %timeit itertuples_at_assign_2()
# %timeit itertuples_iat_assign()

# """
# iat_assign:
# 546 ms ± 8.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# at_assign:
# 566 ms ± 37.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# itertuples_at_assign:
# 228 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# itertuples_at_assign_2:
# 223 ms ± 4.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# itertuples_iat_assign:
# 258 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)"""

In [14]:
# test = Washington_grid.hexes
# w = grid_rings

# %timeit test['integer_form'] = test['grid'].apply(lambda x: x[0]+w+1) + test['grid'].apply(lambda x: x[1]+w+3)*2*w
## 8.32 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

# %timeit test['integer_form']== 2550
## 133 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

# %timeit test['grid']==(25,25,25)
## 1.24 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

In [15]:
# """    def hex_pairing(self):
#         # This uses the fact that every point already has calculated the "id" of the hex in which it falls, using it's coordinates.
#         points = self.points
#         hexes = self.hexes        
#         points['point_id'] = points.index
#         t = tqdm(points.itertuples(), total=points.shape[0])
#         print("Generating list of values within each hex")
        
#         for row in t:
#             id = row[2] # this is the id of the hex it matches
            
#             hex_row = hexes.loc[id]
#             hex_row['total_score'].apply(lambda l: l.append(row.total_score))
#             hex_row['bike_score'].apply(lambda l: l.append(row.bike_score))
#             hex_row['car_score'].apply(lambda l: l.append(row.car_score))
#             hex_row['mass_score'].apply(lambda l: l.append(row.mass_score))
#             hex_row['ride_score'].apply(lambda l: l.append(row.ride_score))
#             hex_row['points_list'].apply(lambda l: l.append(row.point_id))            
            
#             # Selecting the row then adjusting all values runs about twice as fast as looking up each value
            
#             hexes.loc[id, 'total_score'].apply(lambda l: l.append(row.total_score))
#             hexes.loc[id, 'bike_score'].apply(lambda l: l.append(row.bike_score))
#             hexes.loc[id, 'car_score'].apply(lambda l: l.append(row.car_score))
#             hexes.loc[id, 'mass_score'].apply(lambda l: l.append(row.mass_score))
#             hexes.loc[id, 'ride_score'].apply(lambda l: l.append(row.ride_score))
#             hexes.loc[id, 'points_list'].apply(lambda l: l.append(row.point_id))
                
#                 """
